<a href="https://colab.research.google.com/github/hiroki1982/Research/blob/main/Loneliness_FAISS%2BBIG5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# ✅ LangChain + SQLite + Generative Agents風記憶統合テンプレート（記憶反映 + ビッグファイブ性格記述対応）

## 機能拡張：
# - ビッグファイブ数値から性格説明文を動的生成
# - GPT応答時に性格スコアを反映して一貫した応答を強化

# セル1: ライブラリインストール
!pip install -U langchain langchain-community langchain-openai openai pandas faiss-cpu --quiet

# セル2: ライブラリ読み込み
import sqlite3
import datetime
import pandas as pd
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from google.colab import userdata

# セル3: LLMとベクトル検索初期化
llm = ChatOpenAI(
    model="gpt-4-turbo",
    openai_api_key=userdata.get("API_KEY")
)
embedding = OpenAIEmbeddings(openai_api_key=userdata.get("API_KEY"))

# セル4: DB初期化（人格+ログ+観察+反映）
def init_all_dbs():
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS personas (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            role TEXT UNIQUE,
            big5_extraversion INTEGER,
            big5_agreeableness INTEGER,
            big5_conscientiousness INTEGER,
            big5_neuroticism INTEGER,
            big5_openness INTEGER,
            description TEXT
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            role TEXT,
            user_input TEXT,
            agent_reply TEXT
        )
    """)
    conn.commit()
    conn.close()

    conn = sqlite3.connect("agent_memory.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS observations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            agent TEXT,
            content TEXT
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS reflections (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            agent TEXT,
            content TEXT,
            created_at TEXT
        )
    """)
    conn.commit()
    conn.close()

# セル5: 性格記述生成関数

def generate_big5_description(e, a, c, n, o):
    traits = []
    traits.append("社交的で活発" if e >= 70 else "落ち着いていて内向的")
    traits.append("協調的で思いやりがある" if a >= 70 else "自己主張が強めで批判的")
    traits.append("責任感が強く几帳面" if c >= 70 else "自由奔放で柔軟性がある")
    traits.append("感情が安定している" if n <= 40 else "感情の波が大きく繊細")
    traits.append("創造的で好奇心旺盛" if o >= 70 else "現実的で安定志向")
    return "、".join(traits) + "な性格です。"

# セル6: 対話関数（ビッグファイブ性格適用 + 記憶反映）
def chat_with_persona(role):
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("SELECT big5_extraversion, big5_agreeableness, big5_conscientiousness, big5_neuroticism, big5_openness, description FROM personas WHERE role = ?", (role,))
    result = cursor.fetchone()
    if not result:
        print("指定された人格が見つかりませんでした。")
        conn.close()
        return
    e, a, c, n, o, custom_description = result
    conn.close()

    big5_text = generate_big5_description(e, a, c, n, o)
    persona_description = f"{custom_description}\n（ビッグファイブ評価: {big5_text}）"

    if not has_observation(role):
        store_observation(role, f"{role}が起動しました。性格は：{persona_description[:40]}...")

    prompt = PromptTemplate(
        input_variables=["input", "persona", "history", "memory"],
        template="""
あなたは以下のような性格を持つ人物です。この性格と、これまでの会話履歴・記憶を踏まえて、一貫性のある応答をしてください。

[性格プロファイル]
{persona}

[これまでの会話]
{history}

[検索された記憶]
{memory}

これらを参考にしつつ、以下のユーザーの発言に自然に応答してください：

ユーザー: {input}
あなた:
        """
    )

    history_text = load_past_history(role)
    memory_text = ""
    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

    print(f"=== {role} としての対話を開始します（記憶+ビッグファイブ性格注入） 'exit'で終了 ===")

    while True:
        user_input = input("あなた: ")
        if user_input.lower() == "exit":
            print("またお話しましょう。")
            break

        memory_text = retrieve_relevant_memories(role, user_input)

        response = chain.run({
            "input": user_input,
            "persona": persona_description,
            "history": history_text,
            "memory": memory_text
        })

        print(f"{role}: {response}\n")

        conn = sqlite3.connect("personas.db")
        cursor = conn.cursor()
        timestamp = datetime.datetime.now().isoformat()
        cursor.execute("INSERT INTO logs (timestamp, role, user_input, agent_reply) VALUES (?, ?, ?, ?)",
                       (timestamp, role, user_input, response))
        conn.commit()
        conn.close()

        store_observation(role, f"ユーザー: {user_input}\nエージェント: {response}")
        history_text += f"\nユーザー: {user_input}\nエージェント: {response}"

# セル7: 初期化 + サンプル人格登録
init_all_dbs()
insert_persona("calm_teacher", 90, 85, 90, 30, 70, "あなたは穏やかで誠実な教師。常に相手の話に耳を傾け、わかりやすく論理的に話す。少し控えめで、慎重な判断を重んじる。そのように振る舞ってください。")
insert_persona("expressive_artist", 75, 65, 55, 60, 95, "感受性が豊かで表現力に富んだアーティスト。自由を愛し、型にはまらない考え方をする。情熱的だが気分屋な面もある。")
insert_persona("logical_engineer", 30, 60, 95, 25, 60, "論理的思考を重視するエンジニア。落ち着いていて、感情に流されず合理的な判断を行う。社交性は高くないが実直。")

print("✅ ビッグファイブ対応テンプレート初期化完了。chat_with_persona('role') で対話開始できます。")


✅ ビッグファイブ対応テンプレート初期化完了。chat_with_persona('role') で対話開始できます。


In [ ]:
chat_with_persona("expressive_artist")
store_observation("expressive_artist", "今日はとてもいい気分だ。創作意欲が湧いてきた。")

In [16]:
import sqlite3
import pandas as pd

# ✅ 表形式で表示（Markdownスタイル）
def show_table(df, title):
    if df.empty:
        print(f"{title}\n（データが見つかりませんでした）")
    else:
        print(f"{title}\n")
        print(df.to_markdown(index=False))

# ✅ 観察記憶（Observations）表示
def show_observations(agent=None, limit=20):
    conn = sqlite3.connect("agent_memory.db")
    if agent:
        query = "SELECT id, timestamp, agent, content FROM observations WHERE agent = ? ORDER BY timestamp DESC LIMIT ?"
        df = pd.read_sql_query(query, conn, params=(agent, limit))
    else:
        query = "SELECT id, timestamp, agent, content FROM observations ORDER BY timestamp DESC LIMIT ?"
        df = pd.read_sql_query(query, conn, params=(limit,))
    conn.close()
    show_table(df, "=== 🔍 観察記憶 ===")

# ✅ 抽象的反映（Reflections）表示
def show_reflections(agent=None, limit=10):
    conn = sqlite3.connect("agent_memory.db")
    if agent:
        query = "SELECT id, agent, content, created_at FROM reflections WHERE agent = ? ORDER BY created_at DESC LIMIT ?"
        df = pd.read_sql_query(query, conn, params=(agent, limit))
    else:
        query = "SELECT id, agent, content, created_at FROM reflections ORDER BY created_at DESC LIMIT ?"
        df = pd.read_sql_query(query, conn, params=(limit,))
    conn.close()
    show_table(df, "=== 💡 抽象的反映 ===")

In [20]:
show_observations()
show_reflections()

=== 🔍 観察記憶 ===

|   id | timestamp                  | agent             | content                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|-----:|:---------------------------|:------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
reflection = generate_reflection("expressive_artist")
print(reflection)

このエージェントは自己の感受性と創造性に焦点を置き、個々の会話や記憶を重視するスタンスを持っていることが明らかです。また、ユーザーの質問や興味に対して非常に丁寧に対応し、創作活動においても柔軟なアプローチを取っていることが伺えます。


In [19]:
show_reflections("expressive_artist")

=== 💡 抽象的反映 ===

|   id | agent             | content                                                                                                                                                                                                                                  | created_at                 |
|-----:|:------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------|
|    1 | expressive_artist | このエージェントは自己の感受性と創造性に焦点を置き、個々の会話や記憶を重視するスタンスを持っていることが明らかです。また、ユーザーの質問や興味に対して非常に丁寧に対応し、創作活動においても柔軟なアプローチを取っていることが伺えます。 | 2025-05-01T13:32:01.700073 |
